# Generate Mid using dedicated service - F#

### Overview
Mid indicators are a commonly used to provide midpoint value from two given inputs.

This sample demonstrates how to request and plot from a dedicated data service the following midpoints:
1. Midprice
2. Midsize

When correlated with a range of indicators, it gives an idea about the market quality.

### Inputs/outputs
Mid indicator requires instrument's identifier and date time intervals as per inputs and returns the midpoints' values.

For each timestamp, the midpoint's value is an average of:
* bid and ask prices for midprice
* bid and ask sizes for midsize

This sample shows how to plot a simple mid graph for basis technical analysis using an open source library. 

### Services used
This sample uses *gRPC requests* in order to retrieve top of the book tick data from the hosted service. The queried endpoint in this script are:
* *TickBookService*: to directly retrieve best Bid/Ask tick data from the server.

### Packages required
1. Systemathics packages:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.Tick.V1*
2. Open source packages
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*
    * *XPlot.Plotly* as per display package

***

# Run Mid sample

### Step 1: Install packages

In [ ]:
#i "nuget: file:///home/jovyan/.nuget/packages/"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Systemathics.Apis"
#r "nuget: XPlot.Plotly.Interactive"
#r "nuget: TaskBuilder.fs"

In [ ]:
open Systemathics.Apis.Type.Shared.V1
open Systemathics.Apis.Services.Tick.V1
open Google.Protobuf.WellKnownTypes
open Google.Type
open Grpc.Core
open Grpc.Net.Client
open XPlot.Plotly
open XPlot.Plotly.Interactive

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [ ]:
// Retrieve authentication token
let token = System.Environment.GetEnvironmentVariable("AUTH0_TOKEN")
let api = System.Environment.GetEnvironmentVariable("GRPC_APIS")

// Add token to the request header
let headers = new Metadata()
headers.Add("Authorization", $"Bearer {token}")

// Define API entry
let channel = GrpcChannel.ForAddress($"https://{api}")

In [ ]:
// Display authentication token 
token
|> display

### Step 3: Retrieve data
To request *tick data* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* Tick data request parameters

#### 3.1 Instrument selection

In [ ]:
let ticker = "AAPL"
let exchange = "BATS"

#### 3.2 Time period delimitation

In [ ]:
// Create time intervals
let start = new DateTime(2021, 03, 05)
let end' = new DateTime(2021, 03, 05)


// Build the bars request date interval (we are using Google date time format : we have to cast the dates)
let dateIntervals = 
  new DateInterval(
    StartDate = new Date ( Year = start.Year, Month = start.Month, Day = start.Day ),
    EndDate = new Date ( Year = end'.Year, Month = end'.Month, Day = end'.Day )
  )

// Build the bars request time interval (we are using Google date time format : we have to cast the dates)
// UTC time zone
let timeInterval = 
  new TimeInterval(
    StartTime = new TimeOfDay ( Hours = 15, Minutes = 00, Seconds = 00 ),
    EndTime = new TimeOfDay ( Hours = 16, Minutes = 00, Seconds = 00 ) 
  )

#### 3.3 Request creation
The following code snippets creates *gRPC client*, process request and returns the request reply:

In [ ]:
// Generate constraints based on the previous time selection:
let constraints = new Constraints() 
constraints.DateIntervals.Add(dateIntervals)
constraints.TimeIntervals.Add(timeInterval)

In [ ]:
// Generate the tick book request
let request = new TickBookRequest ( Constraints = constraints )
let identifier = new Identifier ( Exchange = exchange, Ticker = ticker)
request.Identifiers.Add [| identifier |]

In [ ]:
// Instantiate the tick book service
let service = new TickBookService.TickBookServiceClient(channel)

### Step 4: Visualize data

#### 4.1 Retrieve Mid data

In [ ]:
open FSharp.Control.Tasks.V2
open System.Collections.Generic

let iterAsync (t: IAsyncEnumerable<'T>) (f : 'T -> unit) =
    let task = 
        task {
            let e = t.GetAsyncEnumerator()
            let mutable go = true
            try
                let! step = e.MoveNextAsync()
                go <- step
                while go do 
                    f e.Current
                    let! step = e.MoveNextAsync()
                    go <- step
            finally
                e.DisposeAsync().AsTask().Wait()
        }
    task.Wait()

// Get the limits
let limits = new List<BookLimit>()
let call = service.TickBook(request, headers)
let e = call.ResponseStream.ReadAllAsync()

let add (t : TickBookResponse ) =
    if (t |> isNull |> not && t.Limit |> isNull |> not) then
        limits.Add t.Limit

iterAsync e add

// Display top of the book
//limits |> display

#### 4.2 Plot Mid

Display Mid over the look back period with the package of your choice.<br><i>`XPlot.Plotly`</i> is used as per open source display package.
Plot a simple line graphs for *midprice* and *midsize* as follows: 

In [ ]:
new Scattergl(
    name = "Mid",
    mode = "lines",
    x = (limits |> Seq.map (fun elt -> elt.TimeStamp.ToDateTime())),
    y = (limits |> Seq.map (fun elt -> (elt.Ask.Price + elt.Bid.Price) / 2.))
)
|> Chart.Plot
|> Chart.WithTitle $"Mid for {ticker}"
|> Chart.WithXTitle "Time"
|> Chart.WithYTitle "Price"
|> Chart.WithWidth 1500
|> Chart.WithHeight 800
|> display